# Algorithm comparation & research

In [ ]:
pip install niapy

In [12]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from niapy.task import Task
from niapy.algorithms.basic import (
    FireflyAlgorithm,
    ParticleSwarmOptimization,
    GreyWolfOptimizer
)
from niapy.problems import Sphere, Rastrigin, Rosenbrock

## Benchmark functions

### Sphere

\[
    f(x) = \sum^{D}_{i=1} x^2_i
\]

* Unimodal
* Convex
* Single global minimum at x = 0
* Smooth gradient, easy convergence

### Rastrigin

\[
    f(x) = 10D + \sum^{D}_{i=1} (x^2_i - 10cos(2\pi x_i))
\]

* Multimodal
* Non-convex
* Many local minima and global minimum at x = 0
* Difficult convergence and challenging for premature convergence

### Rosenbrock

\[
    f(x) = \sum^{D-1}_{i=1} (100(x_{i+1} - x^2_i)^2 + (x_i - 1)^2)
\]

* Unimodal
* Non-convex
* Global minimum at x = (1, 1, 1, \dots, 1)
* Narrow curved valley

### Function parameters

In [13]:
problems = {
    "Sphere": (Sphere, {
        "lower": -5.12,
        "upper": 5.12,
        "max_evals": 10000,
    }),
    "Rastrigin": (Rastrigin, {
        "lower": -5.12,
        "upper": 5.12,
        "max_evals": 30000,
    }),
    "Rosenbrock": (Rosenbrock, {
        "lower": -30,
        "upper": 30,
        "max_evals": 30000,
    }),
}

> **Note:** `niapy` already initializes algorithms with a uniform random distribution by default over the upper and lower bounds defined, so there is no need to apply transformations to the functions due to centering-around-zero bias.

## Algorithms

### Firefly Algorithm

[Yang *et al*., 2009](https://link.springer.com/chapter/10.1007/978-3-642-04944-6_14)

### Particle Swarm Optimization

[Kennedy and Everhart, 1995](https://ieeexplore.ieee.org/document/488968)

### Grey Wolf Optimizer

[Mirjalili *et al*., 2014](https://www.sciencedirect.com/science/article/pii/S0965997813001853?casa_token=Wm5JRdpOZrUAAAAA:Z8DWX5cN2vzQPpJ-9mOd-yUu7GkgJM1Lb_gA1owhmITuD3HKMZZyf3jZ75q43UfmEDc9MWu6)

## Benchmarking

In [ ]:
RESULTS_DIR = "results"
PLOTS_DIR = os.path.join(RESULTS_DIR, "plots")
TABLES_DIR = os.path.join(RESULTS_DIR, "tables")

os.makedirs(PLOTS_DIR, exist_ok=True)
os.makedirs(TABLES_DIR, exist_ok=True)

N_RUNS = 30
POPULATION_SIZE = 30

In [15]:
algorithms = {
    "FA": (FireflyAlgorithm, {
        "population_size": 30,
        "alpha": 0.8,
        "beta0": 1.0,
        "gamma": 0.1
    }),
    "PSO": (ParticleSwarmOptimization, { # Clerc's constriction coefficients
        "population_size": 30,
        "w": 0.729,
        "c1": 1.49445,
        "c2": 1.49445
    }),
    "GWO": (GreyWolfOptimizer, {
        "population_size": 30
    }),
}

In [16]:
def run_single(task, algorithm):
    """
    Run a single instance of an algorithm on a Task.

    Parameters
    ----------
    task : Task
        The optimization task to solve.
    algorithm : Algorithm
        The algorithm instance to run.

    Returns
    -------
    best_position : np.ndarray
        The best solution found.
    best_fitness : float
        The fitness value of the best solution.
    """
    best_position, best_fitness = algorithm.run(task)
    return best_position, best_fitness

In [17]:
import numpy as np

def benchmark_algorithm(algorithm_class, algorithm_params, problem_class, problem_params, n_runs=30, n_points=300, seed_start=0):
    """
    Runs an algorithm multiple times on a problem and collect results.

    Returns
    -------
    fitness_runs : np.ndarray
        Best fitness from each run (length n_runs)
    convergence_avg : np.ndarray
        Averaged convergence curve interpolated onto a common x-axis (length n_points)
    """
    fitness_runs = []
    convergence_curves = []

    evals_max = problem_params["max_evals"]
    x_common = np.linspace(0, evals_max, n_points)

    for run_idx in range(seed_start, seed_start + n_runs):
        np.random.seed(run_idx)

        # Create task and algorithm
        task = Task(
            problem_class(**problem_params),
            max_evals=problem_params["max_evals"],
            max_iters=problem_params["max_evals"] // algorithm_params["population_size"]
        )
        algo = algorithm_class(**algorithm_params)

        # Run single optimization
        best_position, best_fitness = algo.run(task)
        fitness_runs.append(best_fitness)

        # Get convergence data (function evaluations vs best fitness)
        x, y = task.convergence_data(x_axis="evals")

        # Interpolate onto common x-axis
        y_interp = np.interp(x_common, x, y)
        convergence_curves.append(y_interp)

    # Average convergence across runs
    convergence_avg = np.mean(convergence_curves, axis=0)

    return np.array(fitness_runs), convergence_avg

In [23]:
def run_benchmark_loop(problems, algorithms, *, dimension=30, n_runs=30, results_dir="results", n_points=300):
    """
    Runs benchmark for multiple algorithms on multiple problems.
    Saves plots and summary table.
    """

    plots_dir = os.path.join(results_dir, f"{dimension}_d/plots")
    tables_dir = os.path.join(results_dir, f"{dimension}_d/tables")
    os.makedirs(plots_dir, exist_ok=True)
    os.makedirs(tables_dir, exist_ok=True)

    summary_rows = []

    for problem_name, (pclass, pconf) in problems.items():
        plt.figure()
        for algo_name, (algo_class, algo_params) in algorithms.items():
            # Combine dimension with problem config
            problem_params = {"dimension": dimension, **pconf}

            # Run benchmark
            fitness_runs, convergence_avg = benchmark_algorithm(
                algo_class,
                algo_params,
                problem_class=pclass,
                problem_params=problem_params,
                n_runs=n_runs,
                n_points=n_points
            )

            # Save summary statistics
            summary_rows.append({
                "Problem": problem_name,
                "Algorithm": algo_name,
                "Mean fitness": np.mean(fitness_runs),
                "Std fitness": np.std(fitness_runs),
                "Best fitness": np.min(fitness_runs),
                "Worst fitness": np.max(fitness_runs),
            })

            # Plot convergence
            plt.plot(np.linspace(0, problem_params["max_evals"], n_points), convergence_avg, label=algo_name)

        plt.title(f"{problem_name} – Convergence")
        plt.xlabel("Function evaluations")
        plt.ylabel("Best fitness")
        plt.yscale("log")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(os.path.join(plots_dir, f"{problem_name}_convergence.png"))
        plt.close()

    # Save summary table
    df_summary = pd.DataFrame(summary_rows)
    df_summary.to_csv(os.path.join(tables_dir, "summary.csv"), index=False)

In [24]:
run_benchmark_loop(problems, algorithms, dimension=5)

In [25]:
run_benchmark_loop(problems, algorithms, dimension=30)